In [12]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os
import time

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/?ctm=N11'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_data.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")


/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_9346/952689514.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels.db' に保存し、'hotels_data.csv' に書き出しました。


In [7]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os
import time

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/room-dinner00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_9346/2055128154.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels_onsen_info.db' に保存し、'hotels_onsen_info.csv' に書き出しました。


In [19]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/room-nodinner00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info2.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_9346/2646136354.py:55: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels_onsen_info.db' に保存し、'hotels_onsen_info2.csv' に書き出しました。


In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info_hitori.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/room-hitori00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info_hitori3.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_9346/3462684233.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels_onsen_info_hitori.db' に保存し、'hotels_onsen_info_hitori3.csv' に書き出しました。


In [24]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info_dinner.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/plan-dinner00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info_dinner４.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_8278/1274366309.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels_onsen_info_dinner.db' に保存し、'hotels_onsen_info_dinner４.csv' に書き出しました。


In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info_nodinner.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/plan-nodinner00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info_nodinner.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

In [25]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv
import os

# ----------------------------------------------------
# (1) データベースの初期設定: DBファイルとテーブル準備
# ----------------------------------------------------
db_name = 'hotels_onsen_info_nodinner.db'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# 既存のテーブルを削除して再作成
cursor.execute('DROP TABLE IF EXISTS hotels')

# 新しいテーブルを作成
cursor.execute('''
    CREATE TABLE hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        detail_url TEXT,
        onsen_info TEXT
    )
''')

# ----------------------------------------------------
# (2) ランキングサイトのURLを設定してスクレイピング
# ----------------------------------------------------
ranking_url = 'https://www.jalan.net/jalan/doc/kokyuyado-ranking/plan-nodinner00.html'
response = requests.get(ranking_url)
soup = BeautifulSoup(response.content, 'html.parser')

# ホテル名と詳細ページを取得
hotel_links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    text = a_tag.get_text(strip=True)
    if 'uww3001.do' in href and text:
        full_url = 'https://www.jalan.net' + href
        hotel_links.append((text, full_url))

# ----------------------------------------------------
# (3) 各ホテルページから温泉情報を取得しDBに格納
# ----------------------------------------------------
for hotel_name, detail_url in hotel_links:
    time.sleep(2)  # サーバ負荷を軽減するために2秒の遅延を追加
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
    
    # 温泉情報を検索
    onsen_element = detail_soup.find('th', text='温泉')
    if onsen_element:
        onsen_info = onsen_element.find_next_sibling('td')
        if onsen_info:
            onsen_text = onsen_info.get_text(strip=True)
        else:
            onsen_text = '不明'
    else:
        onsen_text = '不明'
    
    # データベースに保存
    cursor.execute('''
        INSERT INTO hotels (name, detail_url, onsen_info)
        VALUES (?, ?, ?)
    ''', (hotel_name, detail_url, onsen_text))

conn.commit()

# ----------------------------------------------------
# (4) DBからデータを読み込んでCSVファイルに出力
# ----------------------------------------------------
# CSVファイル名
csv_file = 'hotels_onsen_info_nodinner5.csv'

# 既存のCSVファイルがあれば削除（必要に応じて処理を変更）
if os.path.exists(csv_file):
    os.remove(csv_file)

# DBからデータ取得
cursor.execute('SELECT name, detail_url, onsen_info FROM hotels')
rows = cursor.fetchall()

# CSVに書き込み
with open(csv_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダー
    writer.writerow(['Hotel Name', 'Detail URL', 'Onsen Info'])
    # データ
    writer.writerows(rows)

conn.close()

print(f"スクレイピングしたホテル情報を '{db_name}' に保存し、'{csv_file}' に書き出しました。")

/var/folders/17/t_n64d8d39s99ps2q3b67hmr0000gn/T/ipykernel_8278/732816481.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  onsen_element = detail_soup.find('th', text='温泉')


スクレイピングしたホテル情報を 'hotels_onsen_info_nodinner.db' に保存し、'hotels_onsen_info_nodinner5.csv' に書き出しました。
